# Display The Data


In [ ]:
from lib.data.dataset import DPHMDataset
import torch
from lib.data.loader import load_intrinsics
from lib.renderer import Camera
import matplotlib.pyplot as plt
from lib.data.preprocessing import point2normal, biliteral_filter
from torchvision.transforms import v2
from torchvision.io import decode_image
from pathlib import Path
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor
import cv2
from lib.utils.visualize import load_pcd
import open3d as o3d


def normal_to_normal_image(normal, mask):
    normal_image = (((normal + 1) / 2) * 255).to(torch.uint8)
    normal_image[~mask] = 255
    return normal_image


def depth_to_depth_image(depth):
    return (depth.clip(0, 1) * 255).to(torch.uint8)


depth_factor: float = 1000
mask_threshold: float = 0.6
idx: int = 48
data_dir = "/home/borth/GuidedResearch/data/ali_kocal_mouthmove"
flame_dir = "/home/borth/GuidedResearch/checkpoints/flame2023_no_jaw"

# load the camera stats
K = load_intrinsics(data_dir=data_dir, return_tensor="pt")
camera = Camera(K=K, width=1920, height=1080, scale=1, device="cpu")

# load the color image
path = Path(data_dir) / "color" / f"{idx:05}.png"
color = pil_to_tensor(Image.open(path)).permute(1, 2, 0)

# load the depth image and transform to m
path = Path(data_dir) / "depth" / f"{idx:05}.png"
img = Image.open(path)
raw_depth = pil_to_tensor(img).to(torch.float32)[0]
depth = raw_depth / depth_factor  # (H,W)

# select the foreground based on a depth threshold
f_mask = (depth < mask_threshold) & (depth != 0)
depth[~f_mask] = mask_threshold

# convert pointmap to normalmap
point, _ = camera.depth_map_transform(depth)
normal, n_mask = point2normal(point)

# create the final mask based on normal and depth
mask = f_mask & n_mask

# mask the default values
color[~mask] = 255
normal[~mask] = 0
depth[~mask] = 0

# create the point maps
depth = biliteral_filter(
    image=depth,
    dilation=50,
    sigma_color=150,
    sigma_space=150,
)
point, _ = camera.depth_map_transform(depth)
point[~mask] = 0

# smooth the normal maps
normal = biliteral_filter(
    image=normal,
    dilation=50,
    sigma_color=250,
    sigma_space=250,
)

normal_image = normal_to_normal_image(normal, mask)
depth_image = depth_to_depth_image(depth)

print(point.shape, point.dtype)
print(normal.shape, normal.dtype)
print(color.shape, color.dtype)
print(mask.shape, mask.dtype, mask.sum(), (~mask).sum())
plt.imshow(normal_image)
# pcd = load_pcd(point.reshape(-1, 3), color=[0,0,255])
# o3d.visualization.draw_plotly([pcd])

In [ ]:
from lib.data.dataset import DPHMPointDataset
import matplotlib.pyplot as plt

idx = 0

new_dataset = DPHMPointDataset(
    data_dir="/home/borth/GuidedResearch/data/christoph_mouthmove",
    scale=8,
    sequence_length=100,
)
new_item = new_dataset[idx]

old_dataset = DPHMPointDataset(
    data_dir="/home/borth/GuidedResearch/data/dphm_christoph_mouthmove",
    scale=8,
    sequence_length=100,
)
old_item = old_dataset[idx]

plt.imshow(new_item["normal"])
plt.show()

plt.imshow(new_item["mask"])
plt.show()

plt.imshow(old_item["normal"].detach().cpu())
plt.show()

plt.imshow(old_item["mask"])
plt.show()

In [ ]:
from lib.utils.visualize import load_pcd
import open3d as o3d

i = 10

m = new_item["mask"]
n = new_item["normal"]
p = new_item["point"]
n[~m] = 1.0
plt.imshow(n)
plt.show()

new_pcd = load_pcd(p[m], color=[255, 0, 0])
o3d.visualization.draw_plotly([new_pcd])

m = old_item["mask"]
n = old_item["normal"]
p = old_item["point"].detach().cpu()
n[~m] = 1.0
plt.imshow(n)
plt.show()

old_pcd = load_pcd(p[m], color=[255, 0, 0])
o3d.visualization.draw_plotly([old_pcd])

In [ ]:
from torchvision.transforms import v2
from pathlib import Path
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor
data_dir = "/home/borth/GuidedResearch/data/dphm_christoph_mouthmove"

# load the camera stats
K = load_intrinsics(data_dir=data_dir, return_tensor="pt")
camera = Camera(K=K, width=1920, height=1080, scale=1, device="cpu")

path = "/home/borth/GuidedResearch/data/christoph_mouthmove/depth/00000.png"
scale = 8
img = Image.open(path)
raw_depth = pil_to_tensor(img).to(torch.float32)[0]
depth = raw_depth / 1000  # (H,W)

f_mask = (depth < 0.6) & (depth != 0)
depth[~f_mask] = 0.0

# depth = biliteral_filter(
#     image=depth,
#     dilation=1,
#     sigma_color=150,
#     sigma_space=150,
# )

point, _ = camera.depth_map_transform(depth)
size = (int(camera.height / scale), int(camera.width / scale))
down_point = v2.functional.resize(
    inpt=point.permute(2, 0, 1),
    size=size,
).permute(1, 2, 0)

image = v2.functional.resize(
    inpt=f_mask.to(torch.float32).unsqueeze(0),
    size=size,
)
down_mask = image[0] == 1.0

old_pcd = load_pcd(down_point[down_mask], color=[255, 0, 0])
o3d.visualization.draw_plotly([old_pcd])

In [ ]:
m = new_item["mask"]
n = new_item["normal"]
p = new_item["point"]
new_pcd = load_pcd(p[down_mask], color=[255, 0, 0])
o3d.visualization.draw_plotly([new_pcd])

In [ ]:
from lib.utils.visualize import load_pcd
import open3d as o3d
import torch

i = 0
p = new_item["point"]
pcd1 = load_pcd(new_item["point"][], color=[255, 0, 0])
o3d.visualization.draw_plotly([pcd1])

In [ ]:
old_item["point"][..., 2].min(), new_item["point"][..., 2].min()

In [ ]:
import torch

nm = torch.linalg.vector_norm(new_item["normal"], dim=-1)

# x = new_item["nomral"} / nm
# x = torch.nan_to_num(x, 0.0)
# torch.linalg.vector_norm(x, dim=-1).max()

x = torch.nn.functional.normalize(new_item["normal"], dim=-1)
# nm = torch.linalg.vector_norm(new_item["normal"], dim=-1)
# nm[new_item["mask"]].min()
plt.imshow(x)
plt.show()


plt.imshow(new_item["normal"])
plt.show()

In [ ]:
mask].min()

In [ ]:
pV_item["mask"].sum()

# Medipipe Landmarks


In [ ]:
from lib.data.loader import (
    load_mediapipe_image,
    load_mediapipe_landmark_2d,
    load_mediapipe_landmark_3d,
)
from lib.model.flame.utils import load_static_landmark_embedding
from lib.renderer import Camera

path = f"/home/borth/GuidedResearch/data/ali_kocal_mouthmove/landmark/00000.pt"
landmark = torch.load(path)

path = f"/home/borth/GuidedResearch/data/ali_kocal_mouthmove/landmark/00000.pt"

# mediapipe_landmarks_3d = load_mediapipe_landmark_3d(
#     data_dir, idx=idx, return_tensor="pt"
# )
# print(f"{mediapipe_landmarks_3d.shape=}")
# print(mediapipe_landmarks_3d[:5, :])

flame_landmarks = load_static_landmark_embedding(flame_dir)
media_idx = flame_landmarks["lm_mediapipe_idx"]

# camera.unproject_points(mediapipe_landmarks_2d)
# landmark[:, 0] *= camera.width
# landmark[:, 1] *= camera.height
# u = landmark[:, 0]
# v = landmark[:, 1]
# landmarks = point[v, u]

plt.scatter(u, v, c="red", s=1.0)
plt.imshow(color)

In [ ]:
camera.screen_transform(landmark)

In [ ]:
camera.width

In [ ]:
u

In [ ]:
landmark

In [ ]:
path = f"/home/borth/GuidedResearch/data/ali_kocal_mouthmove/landmark/00000.pt"
lm1 = torch.load(path)
pcd1 = load_pcd(lm1, color=[255, 0, 0])
o3d.visualization.draw_plotly([pcd1])

In [ ]:
lm1

In [ ]:
from lib.utils.visualize import load_pcd
import open3d as o3d
import torch

i = 25
path = f"/home/borth/GuidedResearch/logs/2024-09-12/07-11-32_optimize/init/batch_landmark/00110/00000.pt"
lm1 = torch.load(path)
pcd1 = load_pcd(lm1, color=[255, 0, 0])

path = f"/home/borth/GuidedResearch/logs/2024-09-12/07-11-32_optimize/init/render_landmark/00110/00000.pt"
lm2 = torch.load(path)
pcd2 = load_pcd(lm2, color=[0, 0, 255])

o3d.visualization.draw_plotly([pcd1, pcd2])

In [ ]:
# path = "/home/borth/GuidedResearch/data/christoph_mouthmove/cache/8_point/00110.pt"
# x = torch.load(path).reshape(-1, 3)
path = "/home/borth/GuidedResearch/data/christoph_mouthmove/landmark/00110.pt"
x = torch.load(path)
pcd = load_pcd(x, color=[0, 0, 255])
o3d.visualization.draw_plotly([pcd])

In [ ]:
from lib.utils.visualize import load_pcd
import open3d as o3d
import torch

i = 0
path = f"/home/borth/GuidedResearch/data/christoph_mouthmove/cache/8_point/{i:05}.pt"
lm1 = torch.load(path).reshape(-1, 3)
path = f"/home/borth/GuidedResearch/data/christoph_mouthmove/cache/8_mask/{i:05}.pt"
mask = torch.load(path).reshape(-1).bool()
pcd1 = load_pcd(lm1[mask], color=[255, 0, 0])
o3d.visualization.draw_plotly([pcd1])

In [ ]:
import torch

for i in range(100):
    path = (
        f"/home/borth/GuidedResearch/data/christoph_mouthmove/landmark_mask/{i:05}.pt"
    )
    x = torch.load(path)
    if x.sum() != 105:
        print(i)

In [ ]:
import matplotlib.pyplot as plt

path = f"/home/borth/GuidedResearch/data/christoph_mouthmove/cache/8_mask/{i:05}.pt"
mask = torch.load(path).bool()
plt.imshow(mask)

In [ ]:
mask.shape

In [ ]:
flame_landmarks = load_static_landmark_embedding(flame_dir)
flame_landmarks.keys()

In [ ]:
from lib.data.loader import (
    load_mediapipe_landmark_2d,
    load_mediapipe_landmark_3d,
    load_intrinsics,
)

import io
from PIL import Image
import torch
import matplotlib.pyplot as plt
import numpy as np

data_dir = "/home/borth/GuidedResearch/data/dphm_christoph_mouthmove"

face_idx = 0
# face_idx = 110
# normal = load_normal(data_dir, image_idx, return_tensor="np")
# normal = (((normal + 1) / 2) * 255).astype(np.uint8)
K = load_intrinsics(data_dir=data_dir, return_tensor="pt")
# face = vertices[:, flame.faces][0][face_idx]
# pixel = camera2pixel(face, K["fx"], K["fy"], K["cx"], K["cy"])
# pixel = camera2pixel(landmarks[0], K["fx"], K["fy"], K["cx"], K["cy"])

# points = media
# xc = points[:, 0]
# yc = points[:, 1]
# zc = points[:, 2]

# us = cx + fx * (xc / zc)
# vs = cy + fy * (yc / zc)

# pixels = (K.to("cuda") @ points.T).T
path = "/home/borth/GuidedResearch/data/christoph_mouthmove/color/00000.png"
color = np.asarray(Image.open(path))


plt.axis("off")
plt.imshow(color)


# draw all of the lm on the screen
# x, y, _ = pixel[0]

W = 1920
H = 1080
u = (mediapipe_landmarks_2d[:, 0] * W).astype(np.int64)
v = (mediapipe_landmarks_2d[:, 1] * H).astype(np.int64)
plt.scatter(u, v, c="red", s=0.5)


# for lm in mediapipe_landmarks_2d:
#     plt.scatter(
#         int(W * lm[0]), int(H * lm[1]), c="red", s=0.5
#     )  # Drawing a red point for each landmark
# plt.scatter(int(pixel[0,0]), int(pixel[0,1]), c="red", s=2)  # Drawing a red point for each landmark
# plt.scatter(int(pixel[1,0]), int(pixel[1,1]), c="blue", s=2)  # Drawing a red point for each landmark
# plt.scatter(int(pixel[2,0]), int(pixel[2,1]), c="green", s=2)  # Drawing a red point for each landmark
# img = load_plt()
# img
# plt.show()
# plt.imshow(img.detach().cpu().numpy())

# Depth

From https://cvg.cit.tum.de/data/datasets/rgbd-dataset/file_formats

The color and depth images are already pre-registered using the OpenNI driver from PrimeSense, i.e., the pixels in the color and depth images correspond already 1:1.

The depth images are scaled by a factor of 1000, i.e., a pixel value of 1000 in the depth image corresponds to a distance of 1 meter from the camera. A pixel value of 0 means missing value/no data.


In [ ]:
from lib.model.flame import FLAME
from lib.renderer.renderer import Renderer
import torch
import matplotlib.pyplot as plt
from lib.utils.mesh import vertex_normals

flame_dir = "/home/borth/GuidedResearch/checkpoints/flame2023"
data_dir = "/home/borth/GuidedResearch/data/dphm_christoph_mouthmove"
scale = 1.0
flame = FLAME(
    flame_dir=flame_dir,
    data_dir=data_dir,
    vertices_mask="full",
).to("cuda")
flame.init_params(
    global_pose=[0.0, 0, 0],
    transl=[0.0, 0.0, -0.5],
)
vertices, landmarks = flame()
renderer = flame.renderer()

# mesh = trimesh.Trimesh(vertices[0].detach().cpu().numpy(), faces=flame.faces.detach().cpu().numpy())
# vn = torch.tensor(mesh.vertex_normals).unsqueeze(0).to(vertices.device)
faces = flame.faces[:, [0, 2, 1]]
vn = vertex_normals(vertices=vertices, faces=faces)
normal, mask = renderer.render(vertices, faces, vn)
# normal, mask = renderer.render(vertices,flame.masked_faces(vertices), vn)
normal_image = renderer.normal_to_normal_image(normal, mask)

In [ ]:
from lib.utils.visualize import load_pcd
import open3d as o3d

pcd = load_pcd(vertices.detach().cpu().numpy()[0])
o3d.visualization.draw_plotly([pcd])

In [ ]:
img.shape

In [ ]:
mediapipe_landmarks_3d

In [ ]:
K

In [ ]:
landmarks = landmarks[0]
landmarks[:, 2] = -landmarks[:, 2]

In [ ]:
landmarks

In [ ]:
normal.shape

In [ ]:
normal[0][int(pixel[0, 1]), int(pixel[0, 0])]

In [ ]:
face

In [ ]:
face

In [ ]:
from lib.model.utils import load_static_landmark_embedding

flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023"
flame_landmarks = load_static_landmark_embedding(flame_dir)
print(flame_landmarks["landmark_indices"][1])
print(flame_landmarks["lmk_face_idx"][1])

In [ ]:
lm_idx = 0

lm3d = load_pipnet_landmark_3d(data_dir, idx=image_idx)[lm_idx]
print(lm3d)

lm2d = load_pipnet_landmark_2d(data_dir, idx=image_idx)[lm_idx]
x, y = lm2d.astype(int)
print(lm2d)

depth = load_depth_masked(data_dir, image_idx, return_tensor="np", depth_factor=1000)
print(depth[y, x])
plt.imshow(depth)

x, y = lm2d.astype(int)
plt.scatter(x, y, c="red", s=10)  # Drawing a red point for each landmark
plt.show()

In [ ]:
import numpy as np
from lib.renderer.camera import load_intrinsics, pixel2camera
from lib.utils.loader import load_depth_masked
import torch
from pathlib import Path
from torchvision.transforms import v2
import matplotlib.pyplot as plt
import torch

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
scale = 0.5

# load the intrinsic
_K = load_intrinsics(data_dir=data_dir, return_tensor="dict")
K = torch.tensor(
    [
        [_K["fx"] * scale, 0.0, _K["cx"] * scale],
        [0.0, _K["fy"] * scale, _K["cy"] * scale],
        [0.0, 0.0, 1.0],
    ]
)

# load the depth image
_depth_masked = load_depth_masked(data_dir, 0, return_tensor="pt")
_H, _W = _depth_masked.shape
H, W = int(_H * scale), int(_W * scale)

# get the mask
_mask = _depth_masked == 0.0
mask = v2.functional.resize(_mask.unsqueeze(0), size=(H, W)).squeeze(0)
mask = ~mask

# get the new size of the depth image
depth_masked = v2.functional.resize(_depth_masked.unsqueeze(0), size=(H, W)).squeeze(0)

# span the pixel indexes
x = torch.arange(W)
y = torch.arange(H)
idx = torch.stack(torch.meshgrid(y, x), dim=-1).flip(-1)

# get the points in camera coordinates, but with the new resolution
points = torch.concat([idx, depth_masked.unsqueeze(-1)], dim=-1)
points[:, :, 0] *= points[:, :, 2]
points[:, :, 1] *= points[:, :, 2]
out = K.inverse() @ points.permute(2, 0, 1).reshape(3, -1)
out = out.reshape(3, points.shape[0], points.shape[1]).permute(1, 2, 0)

# just save the point of the face
cpoints = out[mask]
np.save("temp/out", cpoints.detach().cpu().numpy())
# [depth_masked] * 3
# depth_masked.shape

In [ ]:
from lib.renderer.camera import depth2camera
from lib.utils.loader import load_depth_masked, load_intrinsics
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
depth = load_depth_masked(data_dir=data_dir, idx=0, return_tensor="pt")
scale = 0.1
K = load_intrinsics(data_dir=data_dir, return_tensor="pt", scale=scale)
points = depth2camera(depth, K, scale=scale)
plt.imshow(points[:, :, 2])
points[:, :, 2].min(), points[:, :, 2].max()

In [ ]:
from lib.renderer.camera import camera2normal
import torch

normals = camera2normal(points.unsqueeze(0))

# now show the image
b_mask = normals.sum(-1) == 0
normals_image = (((normals + 1) / 2) * 255).to(torch.uint8)
normals_image[b_mask, :] = 0
plt.imshow(normals_image[0])
# points.unsqueeze(0).shape

In [ ]:
import torch

# https://stackoverflow.com/questions/34644101/calculate-surface-normals-from-depth-image-using-neighboring-pixels-cross-produc/34644939#34644939
# they have (-dz/dx,-dz/dy,1, however we are in camera space hence we need to calculate the gradient in pixel space, e.g. also the delta x and delta y are in camera space.

# make sure that on the boundary is nothing wrong calculated
points[points.sum(-1) == 0] = torch.nan

H, W, C = points.shape
normals = torch.ones_like(points)
normals *= -1

# we calculate the normal in camera space, hence we also need to normalize with the depth information,
# note that the normal is basically on which direction we have the stepest decent.
x_right = torch.arange(2, W)
x_left = torch.arange(0, W - 2)
normals[:, 1:-1, 0] = (points[:, x_right, 2] - points[:, x_left, 2]) / (
    points[:, x_right, 0] - points[:, x_left, 0]
)

y_right = torch.arange(2, H)
y_left = torch.arange(0, H - 2)
normals[1:-1, :, 1] = (points[y_right, :, 2] - points[y_left, :, 2]) / (
    points[y_right, :, 1] - points[y_left, :, 1]
)

# normalized between [-1, 1]
normals = normals / torch.norm(normals, dim=-1).unsqueeze(-1)
normals = torch.nan_to_num(normals, 0)
normals[:1, :, :] = 0
normals[-1:, :, :] = 0
normals[:, :1, :] = 0
normals[:, -1:, :] = 0
b_mask = normals.sum(-1) == 0


# now show the image
normals_image = (((normals + 1) / 2) * 255).to(torch.uint8)
normals_image[b_mask, :] = 0
plt.imshow(normals_image)

In [ ]:
(points[:, x_right, 2] - points[:, x_left, 2]).max()

In [ ]:
import open3d as o3d

import numpy as np

camera = np.load("temp/out.npy").reshape(-1, 3)
points = camera[camera[:, 2] != 0]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_plotly([pcd])

We can see that we have a point in 3D which is:

[-0.051, -0.042, 0.575] (x, y, z)

The coresponding pixel value is:

[878, 480] (x, y)

How do we get from 3D to 2D screen coordinates?

Input:
fx = 914.415
fy = 914.03
cx = 959.598
cy = 547.202
xyz_camera = [-0.051, -0.042, 0.575] (x, y, z_c)

Output:
uvz_pixel = [878.0, 480.0, 0.575] (u, v, z_c)


In [ ]:
from lib.utils.loader import load_pipnet_landmark_3d
from lib.renderer.camera import load_intrinsics, camera2pixel

flame_landmarks = load_static_landmark_embedding(flame_dir)
lm_idx = flame_landmarks["landmark_indices"]

plt.imshow(color)

lm3d = load_mediapipe_landmark_3d(data_dir, idx=image_idx)
K = load_intrinsics(data_dir=data_dir)
lm = camera2pixel(lm3d, **K)
for point in lm[lm_idx]:
    x, y, z = point.astype(int)
    plt.scatter(x, y, c="red", s=1)  # Drawing a red point for each landmark
plt.show()

# Normals and Points in 3D


In [ ]:
from lib.utils.loader import load_normals_3d, load_points_3d
import open3d as o3d

normals = load_normals_3d(data_dir=data_dir, idx=0)
print(f"{normals.shape=}")
print(normals[:5, :])

points = load_points_3d(data_dir=data_dir, idx=0)
print(f"{points.shape=}")
print(points[:5, :])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_plotly([pcd])

In [ ]:
import open3d as o3d
from lib.utils.loader import load_points_3d
from pathlib import Path

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
points = load_points_3d(data_dir=data_dir, idx=0)
print(f"{points.shape=}")
print(points[:5, :])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_plotly([pcd])

In [ ]:
import numpy as np

path = "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/camera/c00_color_extrinsic.txt"
E = np.zeros((4, 4))
E[3, 3] = 1.0
E[:3, :] = np.loadtxt(path).reshape(3, 4)  # extrinsic hence world to camera

# note that the pose is the camera to world, e.g. if flame calls them pose they mean
# that they project from camera to world coordinates, hence the final mesh vertices lives
# in the world coordinate system! This is so important!
# note that this is 4x4
# we need to project the point from camera to world! because the point cloud is in camera
# we can see that because the coordinate system is right-hand where z-axes goes inside and
# y-axes goes down, usually z goes to the camera and y up (see cv2 reference)
pose = np.linalg.inv(E)  # camera to world, hence this is the "pose" they call it that.

points_c_homo = np.zeros((points.shape[0], 4))
points_c_homo[:, 3] = 1.0
points_c_homo[:, :3] = points


points_w_homo = (E @ points_c_homo.T).T

In [ ]:
points_w_homo = (pose[:3, :3] @ points.T).T
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_w_homo)
o3d.visualization.draw_plotly([pcd])

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_w_homo[:, :3])
o3d.visualization.draw_plotly([pcd])

In [ ]:
from pathlib import Path

data_dir = Path("/home/borth/GuidedResearch/data/dphm_kinect")
dataset_names = sorted(list(p.name for p in data_dir.iterdir()))
# min([len(list((p/"depth").iterdir())) for p in data_dir.iterdir()])
dataset_names

In [14]:
train = [
    "ali_kocal_mouthmove",
    "ali_kocal_rotatemouth",
    "aria_talebizadeh_mouthmove",
    "aria_talebizadeh_rotatemouth",
    "arnefucks_mouthmove",
    "arnefucks_rotatemouth",
    "changluo_rotatemouth",
    "christoph_mouthmove",
    "elias_wohlgemuth_mouthmove",
    "felix_mouthmove",
    "honglixu_mouthmove",
    "honglixu_rotatemouth",
    "innocenzo_fulgintl_mouthmove",
    "innocenzo_fulgintl_rotatemouth",
    "leni_rohe_mouthmove",
    "leni_rohe_rotatemouth",
    "madhav_agarwal_mouthmove",
    "madhav_agarwal_rotatemouth",
]

bad = [
    "haoxuan_mouthmove",
    "haoxuan_rotatemouth"
    "felix_rotatemouth",
    "changluo",
]
xs = [s.split("_")[-1] for s in train]
x = sum([1 for x in  xs if x =="mouthmove"])
x
# xs = [s.split("_")[0] for s in train]
# len(set(xs))


24